# **Standard pipeline: analyzing our atac data dataset from seq2science**

In [1]:
import snapatac2 as snap
import scanpy as sc
import pandas as pd

In [ ]:
#Look into fragment.tsv files by looking at them on here

In [1]:
#devcornea_atac.h5ad
#snap.pp.make_fragment_file(bam_file = "/ceph/rimlsfnwi/data/moldevbio/zhou/jarts/cellrangeratac/cellranger-atac-2.0.0/ataclako1/outs/possorted_bam.bam", output_file = 'practice.h5ad', is_paired=True, barcode_tag=None, barcode_regex=None, umi_tag=None, umi_regex=None, shift_left=4, shift_right=- 5, min_mapq=30, chunk_size=50000000)

In [3]:
#Download the fragment file.

# Input files
fragment_file = pd.read_csv('fragments_corrected_count.tsv', sep='\t')
fragment_file.tail()

,chr1,79652,80068,CTAACTTCATTCGTCC
20492372,chrY,56871888,56872278,CCAATGAAGATATGTG
20492373,chrY,56874582,56874647,TACCTCGGTATCCTTT
20492374,chrY,56874582,56874647,TACCTCGGTATCCTTT
20492375,chrY,56874582,56874647,TACCTCGGTATCCTTT
20492376,chrY,56874582,56874647,TACCTCGGTATCCTTT


In [5]:
fragment_file_1 = pd.read_csv('atac_pbmc_5k_nextgem_fragments.tsv', sep='\t', on_bad_lines='skip') #on_bad_lines='skip' added this to fix parse error
fragment_file_1.tail()
#How do you remove such a long section in the beginning of the tsv file, as seen below, look at tail of this file on terminal

In [3]:
#Should we be using this bam file ?
adata = snap.read("/ceph/rimlsfnwi/data/moldevbio/zhou/imarais/data/atac_seq_dev_eye/SRR11692131/final_bam/hg38-SRR11692131.samtools-coordinate.bam")

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: H5Fopen(): unable to open file: file signature not found', /root/.cargo/git/checkouts/anndata-rs-087569d311f7caf2/b63f979/pyanndata/src/anndata.rs:36:56
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Result::unwrap()` on an `Err` value: H5Fopen(): unable to open file: file signature not found

In [7]:
adata

In [2]:
snap.genome.hg38

## Preprocessing

In [ ]:
#backed mode used
# import magic
data = snap.pp.import_data(
    fragment_file,
    genome=snap.genome.hg38,
    file="pbmc.h5ad",
    sorted_by_barcode=False,
    low_memory=False,
)
data

To identify usable/high-quality cells, we can plot TSS enrichment against number of unique fragments for each cell

In [ ]:
#it doesnt run, this function has an issue
#snap.pl.tsse(data, interactive=False)

According to the above plot, we decided to choose minimum TSS enrichment of 10 and minimum number of fragments of 5000 to filter the cells.

In [ ]:
snap.pp.filter_cells(data, min_counts=5000, min_tsse=10, max_counts=50000)
data

We next create a cell by bin matrix containing insertion counts across genome-wide 500-bp bins

In [ ]:
snap.pp.add_tile_matrix(data)

In [ ]:
data

The default feature selection algorithm selects variable features. You can pass filter list to the function such as blacklist or whitelist.

In [ ]:
snap.pp.select_features(data)

## Doublet removal

In [ ]:
#apply a customized scrublet algorithm to identify potential doublets

snap.pp.scrublet(data)

In [ ]:
snap.pp.call_doublets(data)
snap.pl.scrublet(data, interactive=False)

Note the .subset() method is available only in backed mode. To subset an in-memory AnnData object, use: data = data[~data.obs["is_doublet"], :] instead.

In [ ]:
data.subset(~data.obs["is_doublet"])
data